## Import libraries and define helper functions

In [ ]:
from tqdm import tqdm_notebook
import numpy as np
import pickle
import pandas as pd
import configparser
import psycopg2
import os
import ast
from sklearn.preprocessing import MinMaxScaler
import networkx as nx
import random
import time
import math
import datetime
import csv
import statistics
import copy
from collections import defaultdict
from statistics import harmonic_mean

start = time.time()

# Return the arithmetic mean of a list of numbers.
def calculate_mean(numbers):
    return sum(numbers) / len(numbers)

# Return the sample standard deviation of a list of numbers.
def calculate_standard_deviation(numbers):
    return statistics.stdev(numbers)

# Merge multiple RWR score lists and compute a harmonic mean per gene.
def combine_lists(full_list):
    combined = defaultdict(list)
    
    for sublist in full_list:
        for item in sublist:
            entrez_id, rwr_score = item
            combined[entrez_id].append(rwr_score)
    
    result = []
    for entrez_id, scores in combined.items():
        if len(scores) == 1:
            result.append([entrez_id, scores[0]])
        else:
            harmonic_mean_score = harmonic_mean(scores)
            result.append([entrez_id, harmonic_mean_score])
    
    return result

def compound_target(compoundid):
    
    compound_gene1=conn_cur.fetchall()

    compound_gene=set(compound_gene1)
    compound_gene=list(compound_gene)
    
    direct_target_gene = [] 
    for i in range(len(compound_gene)):
        interaction_gene = compound_gene[i][0]
        if interaction_gene in entrez_list:
            direct_target_gene.append(interaction_gene)        
        else:
            continue

    direct_seed_node = [] 
    for entrez in direct_target_gene:
        node_ = node_index.get(entrez) 
        direct_seed_node.append(node_)
    
    num1=len(direct_target_gene) 
    
    var_name1 = f'{compoundid}_num1'

    globals()[var_name1] = num1

    return direct_target_gene

# Run RWR after sampling random genes equal in count to real targets.
def rwr_compound_target(count,entrez_list):
    
    num1=directed_target_gene_count_list[count]
    
    random_interaction_gene1 = random.sample(entrez_list, num1)

    direct_seed_node = [] 
    for entrez in random_interaction_gene1:
        node_ = node_index.get(entrez)
        direct_seed_node.append(node_)

    initialized = RWR_initializing(G, seed_nodes=direct_seed_node, is_weighted=False) 
    
    rwr=RWR(initialized=initialized, prob=0.25, max_iter=100, tol=1.0e-6)
    rwr_mapping_entrez = [[entrez_list[i], rwr[node_index.get(entrez_list[i])]] for i in range(len(entrez_list))]
    return rwr_mapping_entrez

# Build an undirected PPI graph and a node-to-index dictionary.
def create_ppi_network(network_data):
    network_data = network_data.astype({'Entrez Gene Interactor A': str,
                                        'Entrez Gene Interactor B': str})

    symbolA = network_data.loc[:, 'Entrez Gene Interactor A'].to_list()
    symbolB = network_data.loc[:, 'Entrez Gene Interactor B'].to_list()

    symbol_list = symbolA + symbolB 
    symbol_list = set(symbol_list) 
    symbol_list = list(symbol_list)

    node_index = {}
    for i in range(len(symbol_list)):
        node_index[symbol_list[i]] = i

    node_list = node_index.values()
    
    edge_list = network_data[['Entrez Gene Interactor A', 'Entrez Gene Interactor B']].values.tolist()
    for i in range(len(edge_list)):
        edge_list[i][0] = node_index.get(edge_list[i][0])
        edge_list[i][1] = node_index.get(edge_list[i][1])
    
    G = nx.Graph()
    G.add_nodes_from(node_list)
    G.add_edges_from(edge_list)

    print(f'Number of created nodes: {G.number_of_nodes()}')
    print(f'Number of created edges: {G.number_of_edges()}')

    return G, node_index

# Prepare normalized adjacency matrix and initial vectors for RWR.
def RWR_initializing(G, seed_nodes, is_weighted=False):
    norm_A = np.zeros(shape=(len(G), len(G)))
    if is_weighted:  
        for i, neighbor_dict in G.adjacency():
            for j, v in neighbor_dict.items():
                norm_A[i][j] = v.get("weight", 1 / len(neighbor_dict))
    else:  
        for i, neighbor_dict in G.adjacency():  
            for j, v in neighbor_dict.items():  
                norm_A[i][j] = 1 / len(neighbor_dict)
    
    personalization = {node: 1 for node in seed_nodes}
        
    r_0 = np.array([personalization.get(n, 0) for n in range(len(G))]) 
    r_c = np.repeat(1 / len(G), len(G)) 
    return {"norm_A": norm_A, "r_0": r_0, "r_c": r_c, "N": len(G)}

# Perform Random Walk with Restart until convergence or max_iter.
def RWR(initialized, prob=0.25, max_iter=100, tol=1.0e-6):
    if initialized is None:
        raise ValueError('initialized information must be required')

    norm_A = initialized['norm_A']  
    r_0 = initialized['r_0']        
    r_c = initialized['r_c']     
    N = initialized['N']           
    for iteration in range(max_iter):
        r_prev = r_c
        r_c = prob * r_c @ norm_A + (1 - prob) * r_0 
        err = np.absolute(r_c - r_prev).sum()
        if err < N * tol:
            return r_c
    return "NotConverged"

In [ ]:
network_data = pd.read_csv("Dataset/BIOGRID-ORGANISM-Homo_sapiens-4.4.228, 9606-9606.csv", encoding="cp949")
G, node_index = create_ppi_network(network_data=network_data)  
entrez_list = list(node_index.keys())

gene_phenotype_file = 'Dataset/ENTREZ_GENE_ID2GOTERM_BP_DIRECT_2.csv'
if os.path.isfile(gene_phenotype_file): 
    gene_phenotype_df = pd.read_csv(gene_phenotype_file, encoding='UTF-8', converters={'Phenotype': ast.literal_eval}) 

phenotype_list = []
for phen_list in gene_phenotype_df['Phenotype']:
    for phen in phen_list:
        if phen not in phenotype_list:
            phenotype_list.append(phen)

## Define Compound groups

In [ ]:
number=1
compound_text=[]
text=["adenine", "canavanin", "malate", "isoleucine", "phenylalanine", "tyrosine", "isocitrate", "glutamine", "tryptophan","L-malate", 
      "theogallin", "trans-aconitate", "cis-aconitate", "threonate", "benzoic acid", "choline", "citric acid", "coumarin", "phloroglucinol", "gallic acid", 
      "citrullin", "pipecolic acid", "vitamin B", "pantothenic acid", "DL-valine", "vanillin", "theophylline", "caffeine", "protocatechuic aldehyde", "adenosine", 
      "EGCG", "gallocatechin", "N-acetylglutamate", "loliolide", "procyanidin B2", "3H-proline", "dihydromyricetin", "beta-alanine betaine", "epicatechin gallate (ECG", "5'-methylthioadenosine", 
      "epiafzelechin", "ferulic acid", "p-coumaric acid", "caffeic acid", "chlorogenic acid", "quercetin", "alpha-isopropylmalate", "neochlorogenic acid", "rutin", "hyperoside", 
      "4-p-coumaroylquinic acid", "astragalin", "quercetin 7-O-glucoside", "p-coumaroylquinic acid", "procyanidin B1"]
compound_text.append(text)

number=2
text=["gamma-aminobutyric acid (GABA", "canavanin", "sucrose", "folate", "guanosine", "vanillic acid", "ononin", 
      "formononetin", "calycosin", "sissotrin", "afrormosin", "pratensein"]
compound_text.append(text)

number=3
text=["protocatechuic acid", "gamma-aminobutyric acid (GABA", "choline", "citric acid", "malate", "asparagine", 
      "quinic acid", "maltol", "nodakenetin", "nodakenin", "columbianetin", "decursin", "ferulic acid", "caffeic acid",
      "aegelinol", "cis-chlorogenic acid", "chlorogenic acid", "esculetin"]
compound_text.append(text)

number=4
text=["piperine", "piperanine", "piperlonguminine", "retrofractamide B", "dehydropipernonaline", "Methyl piperate", "pipernonaline", "Retrofractamide A", "retrofractamide C"]
compound_text.append(text)

number=5
text=["apocynin", "resacetophenone", "4-hydroxyacetophenone", "2,5-dihydroxyacetophenone", "physcion"]
compound_text.append(text)

number=6
text=["protocatechuic acid", "p-hydroxybenzaldehyde", "choline", "gluconate", "myo-inositol", "thymine", "uracil", "vanillin", "vanillic acid", "protocatechuic aldehyde", "trans-aconitate", "ferulic acid", "caffeic acid", "chlorogenic acid", "linoleic acid", "neochlorogenic acid", "xanthatin", "cynarin", "cynarine"]
compound_text.append(text)

number=7
text=["atractylenolide III", "vanillic acid", "esculetin", "elemicin", "adenosine"]
compound_text.append(text)


## Quantifying Compound Effects through Random Walk with Restart and Pathway Score Inference

In [ ]:
number=0
for text in compound_text:
    number=number+1
    print(number,"_Compound group")
    Compoundid_list=[]
    for compound in text:

        result = conn_cur.fetchone()
        if result:
            Compoundid_list.append(result[0])
        else:
            Compoundid_list.append(None)  # Append None when the compound ID is not found

    if not os.path.exists('Results/Direct_GOBP/Compound group_{}'.format(number)):
        os.makedirs('Results/Direct_GOBP/Compound group_{}'.format(number)) 

    Compound_list=copy.deepcopy(Compoundid_list)

    global directed_target_gene_count_list
    directed_target_gene_count_list=[]  # List storing the number of direct target genes for each input compound

    direct_target_gene_list=[] # Direct target genes for each compound

    for c in range(len(Compound_list)):

        compoundid1=Compound_list[c]

        direct_output_list= compound_target(compoundid1)
        
        direct_target_gene_list=direct_target_gene_list+direct_output_list

    direct_target_gene_list=set(direct_target_gene_list)
    direct_target_gene_list=list(direct_target_gene_list)

    direct_seed_node = [] 
    for entrez in direct_target_gene_list:
        node_ = node_index.get(entrez) 
        direct_seed_node.append(node_)
    
    num1=len(direct_target_gene_list) 
    directed_target_gene_count_list.append(num1)
    print("Number of direct target genes ",num1)    
    
    initialized = RWR_initializing(G, seed_nodes=direct_seed_node, is_weighted=False) 

    rwr = RWR(initialized=initialized, prob=0.25, max_iter=100, tol=1.0e-6)
    rwr_mapping_entrez = [[entrez_list[i], rwr[node_index.get(entrez_list[i])]] for i in range(len(entrez_list))]

    rwr_result = pd.DataFrame(data=rwr_mapping_entrez, columns=['Entrez ID', 'RWR_score'])
    rwr_result['Entrez ID'] = rwr_result['Entrez ID'].astype(int)
    rwr_result = pd.merge(rwr_result, gene_phenotype_df, how='outer')
    rwr_result = rwr_result.dropna(subset=['RWR_score'])
    rwr_result['Phenotype'].loc[rwr_result['Phenotype'].isnull()] = rwr_result['Phenotype'].loc[
        rwr_result['Phenotype'].isnull()].apply(lambda x: [])
    rwr_result = rwr_result.sort_values(by='RWR_score', ascending=False)

    phenotype_rwr_score_dict = {}
    for phen in phenotype_list:
        phenotype_rwr_score_dict[phen] = 0    

    for i in range(len(rwr_result)):
        rwr_score = rwr_result['RWR_score'].iloc[i]
        if rwr_result['Phenotype'].iloc[i]:
            for phen in rwr_result['Phenotype'].iloc[i]:
                phenotype_rwr_score_dict[phen] += rwr_score

    temp=list(phenotype_rwr_score_dict.keys())
    for i in temp:
        if np.isnan(phenotype_rwr_score_dict[i])==True:
            phenotype_rwr_score_dict.pop(i)

    Phenotype_score_Phenotype=list(phenotype_rwr_score_dict.keys())
    Phenotype_score_Score=list(phenotype_rwr_score_dict.values())

    # Save phenotype scores
    inferred_phenotype2 = pd.DataFrame(data=[pair for pair in zip(list(phenotype_rwr_score_dict.keys()),
                                                                 list(phenotype_rwr_score_dict.values()))],
                                      columns=['Phenotype', 'Network_score'])
    inferred_phenotype2 = inferred_phenotype2.sort_values(by='Network_score', ascending=False)
    inferred_phenotype2.to_csv('Results/Direct_GOBP/Compound group_{}/{}_phenotype_score.csv'.format(number,number),index=False, encoding='UTF-8')

    output_dict={}
    #----------------------------------------------------------------Statistical Tests
    for h in tqdm_notebook(range(1000)):
        random_interaction_gene1 = random.sample(entrez_list, num1)
        
        direct_seed_node = [] 
        for entrez in random_interaction_gene1:
            node_ = node_index.get(entrez)
            direct_seed_node.append(node_)
    
        initialized = RWR_initializing(G, seed_nodes=direct_seed_node, is_weighted=False) 
    
        rwr=RWR(initialized=initialized, prob=0.25, max_iter=100, tol=1.0e-6)
        rwr_mapping_entrez = [[entrez_list[i], rwr[node_index.get(entrez_list[i])]] for i in range(len(entrez_list))]

        rwr_result = pd.DataFrame(data=rwr_mapping_entrez, columns=['Entrez ID', 'RWR_score'])
        rwr_result['Entrez ID'] = rwr_result['Entrez ID'].astype(int)
        rwr_result = pd.merge(rwr_result, gene_phenotype_df, how='outer')
        rwr_result = rwr_result.dropna(subset=['RWR_score'])
        rwr_result['Phenotype'].loc[rwr_result['Phenotype'].isnull()] = rwr_result['Phenotype'].loc[
            rwr_result['Phenotype'].isnull()].apply(lambda x: [])
        rwr_result = rwr_result.sort_values(by='RWR_score', ascending=False)
        
        phenotype_rwr_score_dict = {}
        for phen in phenotype_list:
            phenotype_rwr_score_dict[phen] = 0
            
        for i in range(len(rwr_result)):
            rwr_score = rwr_result['RWR_score'].iloc[i]
            if rwr_result['Phenotype'].iloc[i]:
                for phen in rwr_result['Phenotype'].iloc[i]:
                    phenotype_rwr_score_dict[phen] += rwr_score

        key_list=list(phenotype_rwr_score_dict.keys())
        for i in key_list:
            if i not in output_dict:
                output_dict[i]=list([phenotype_rwr_score_dict[i]])
            else:
                output_dict[i].append(phenotype_rwr_score_dict[i])

        with open("Results/Direct_GOBP/Compound group_{}/output_dict.pickle".format(number),"wb") as f:
            pickle.dump(output_dict,f)
    print("Completed 1,000 random RWR iterations")
    
    temp2=list(output_dict.keys())
    for i in temp2:
        if np.isnan(output_dict[i][0])==True:
            output_dict.pop(i)

    inferred_phenotype3 = pd.DataFrame(data=[pair for pair in zip(list(output_dict.keys()),
                                                                 list(output_dict.values()))],
                                      columns=['Phenotype', 'Network_score'])
    inferred_phenotype3 = inferred_phenotype3.sort_values(by='Network_score', ascending=False)

    inferred_phenotype3.to_csv('Results/Direct_GOBP/Compound group_{}/{}_P_value.csv'.format(number,number),index=False, encoding='UTF-8')

    P_value_Phenotype=list(output_dict.keys())
    P_value_Score=list(output_dict.values())
    
    #----------------------------------------------------------------z-score  
    count=0
    z_score_list=[]
    outlier_list=[] 
    P_value_list=[]
    rank_number=0
    result_phenotype=[]
    result_phenotype_score=[]

    for i in range(len(P_value_Score)):
        Numberlist=P_value_Score[i].copy()
        tmp1=P_value_Phenotype[i]
        tmp2=Phenotype_score_Phenotype.index(tmp1)
        tmp3=Phenotype_score_Score[tmp2] 
        Numberlist.append(tmp3)
        result_phenotype.append(tmp1)
        result_phenotype_score.append(tmp3)

        mean = calculate_mean(Numberlist)

        std_deviation = calculate_standard_deviation(Numberlist)
        if std_deviation==0:    
            z_score=0
        else:
            z_score=(tmp3-mean)/std_deviation

        z_score_list.append(z_score)
        if z_score >=-2.58 and z_score<=2.58:
            outlier=1
            count=count+1
        else:
            outlier=0 
        outlier_list.append(outlier) 

        Numberlist.sort(reverse=True)

        rank=Numberlist.index(tmp3) 
        if rank<5:
            rank_number=rank_number+1
        P_value_list.append(rank)

    Phenotype_score = pd.DataFrame({
        'Phenotype': result_phenotype,
        'Network_score': result_phenotype_score,
        'z-score': z_score_list,
        'P-value': P_value_list,
    })

    Phenotype_score.to_csv('Results/Direct_GOBP/Compound group_{}/{}_validation.csv'.format(number,number),index=False, encoding='UTF-8')
    
    network_scores = Phenotype_score['Network_score']

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_scores = scaler.fit_transform(network_scores.values.reshape(-1, 1)) 

    Phenotype_score['scaling_score'] = scaled_scores

    def adjust_pvalue(value):
        if value == 999 or value == 1000:
            return 1
        else:
            return (value+1) / 1000

    Phenotype_score['P-value'] = Phenotype_score['P-value'].apply(adjust_pvalue)


    #------------------------------------Merge phenotype scores with UMLS IDs

    file_2= pd.read_csv("Dataset/UMLSID.csv")

    file_3 = pd.merge(Phenotype_score, file_2[['umlsid', 'phenotype','hpoid']], left_on='Phenotype', right_on='phenotype', how='left')

    file_3.drop(columns=['phenotype'], inplace=True)
    Phenotype_score=file_3

    mapping_df = pd.read_excel('Dataset/phenotype mappingv4.xlsx', engine='openpyxl')

    grouped_mapping_df = mapping_df.groupby('UMLSID')['Group'].agg(lambda x: ','.join(x)).reset_index()

    final_df = pd.merge(Phenotype_score, grouped_mapping_df, left_on='umlsid', right_on='UMLSID', how='left')

    final_df.drop(columns=['UMLSID'], inplace=True)

    cols_to_drop = [col for col in final_df.columns if col.startswith("Unnamed")]
    final_df = final_df.drop(columns=cols_to_drop)

    final_df.to_csv('Results/Direct_GOBP/Compound group_{}/{}_validation_with_scaling.csv'.format(number,number), index=False)
    

end = time.time()
sec = (end - start)
result = datetime.timedelta(seconds=sec)
print("Elapsed time : ",result)
